# Imports and choosing test and train area of interest

In [1]:
# Imports
import ee
import pprint
import folium

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [3]:
# Get an area in Brazil using http://geojson.io
test_geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -50.69091796875,
              -9.96885060854611
            ],
            [
              -50.614013671875,
              -9.96885060854611
            ],
            [
              -50.614013671875,
              -9.893098633379571
            ],
            [
              -50.69091796875,
              -9.893098633379571
            ],
            [
              -50.69091796875,
              -9.96885060854611
            ]
          ]
        ]
      }
    }
  ]
}

# Put coordinates into ee format
test_coords = test_geoJSON['features'][0]['geometry']['coordinates']
test_aoi = ee.Geometry.Polygon(test_coords)



train_geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -52.81814575195312,
              -10.668705375726015
            ],
            [
              -52.33612060546875,
              -10.668705375726015
            ],
            [
              -52.33612060546875,
              -10.09867012060338
            ],
            [
              -52.81814575195312,
              -10.09867012060338
            ],
            [
              -52.81814575195312,
              -10.668705375726015
            ]
          ]
        ]
      }
    }
  ]
}


train_coords = train_geoJSON['features'][0]['geometry']['coordinates']
train_aoi = ee.Geometry.Polygon(train_coords)

# Loading Vancutsem and Landsat data

Load data in testing AOI

In [52]:
# Vancutsem data guide available at https://forobs.jrc.ec.europa.eu/TMF/download/TMF_DataUsersGuide.pdf

# the year a pixel is first deforested
test_deforestation_year = ee.ImageCollection('projects/JRC/TMF/v1_2020/DeforestationYear').select("DeforestationYear").filterBounds(test_aoi)

# total number of disturbances
test_intensity = ee.ImageCollection('projects/JRC/TMF/v1_2020/Intensity').select("Intensity").filterBounds(test_aoi)

# transition map: which class each pixel is in
test_transition_map = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_MainClasses').mosaic()
# Remap Vancutsem transition map into something easier to use
# Forest regrowth --> Undisturbed tropical moist forest, all deforested land types in one class (including ongoing deforestation)
test_transition_map = test_transition_map.remap([10,20,30,41,42,43,50,60,70],[0,1,0,2,2,2,2,3,4]).rename("class")
# 0 = Undisturbed, 1 = Degraded, 2 = Deforested, 3 = Water, 4 = Other

# PALSAR SAR data
test_palsar = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR').filterBounds(test_aoi).filterDate('2019-01-01', '2019-12-31')
# check which bands are available in aoi in the most recent image (VV and VH are not available, maybe due to the direction the satellite is pointing)
print("PALSAR bands: " ,test_palsar.sort('system:time_start', False).first().clip(test_aoi).bandNames().getInfo()) 
test_palsar = test_palsar.select('HH')

# Sentinel-1 SAR data (split into two lines because it is very long)
test_sentinel = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(test_aoi).filterDate('2019-01-01', '2019-12-31')
# check which bands are available in aoi in the most recent image (HH and HV are not available, maybe due to the direction the satellite is pointing)
print("Sentinel-1 bands: " ,test_sentinel.sort('system:time_start', False).first().clip(test_aoi).bandNames().getInfo()) 
test_sentinel = test_sentinel.select('VV')



# Landsat 8 data: we only use 2019 data
test_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(test_aoi).filterDate('2019-01-01', '2019-12-31')

# function to compute NDVI
def calculate_NDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# compute NDVI
test_landsat_ndvi = test_landsat.map(calculate_NDVI).select('NDVI')




PALSAR bands:  ['HH', 'HV', 'angle', 'date', 'qa']
Sentinel-1 bands:  ['VV', 'VH', 'angle']


Load data in training aoi

In [53]:
# Copy/pasting code like this is not ideal, but having different variable names at least means data leakage is impossible

# Vancutsem data guide available at https://forobs.jrc.ec.europa.eu/TMF/download/TMF_DataUsersGuide.pdf

# the year a pixel is first deforested
train_deforestation_year = ee.ImageCollection('projects/JRC/TMF/v1_2020/DeforestationYear').select("DeforestationYear").filterBounds(train_aoi)

# total number of disturbances
train_intensity = ee.ImageCollection('projects/JRC/TMF/v1_2020/Intensity').select("Intensity").filterBounds(train_aoi)

# transition map: which class each pixel is in
train_transition_map = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_MainClasses').mosaic()
# Remap Vancutsem transition map into something easier to use
# Forest regrowth --> Undisturbed tropical moist forest, all deforested land types in one class (including ongoing deforestation)
train_transition_map = train_transition_map.remap([10,20,30,41,42,43,50,60,70],[0,1,0,2,2,2,2,3,4]).rename("class")
# 0 = Undisturbed, 1 = Degraded, 2 = Deforested, 3 = Water, 4 = Other

# PALSAR SAR data
train_palsar = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR').filterBounds(train_aoi).filterDate('2019-01-01', '2019-12-31')
# check which bands are available in aoi in the most recent image (VV and VH are not available, maybe due to the direction the satellite is pointing)
print("PALSAR bands: " ,train_palsar.sort('system:time_start', False).first().clip(train_aoi).bandNames().getInfo()) 
train_palsar = train_palsar.select('HH')

# Sentinel-1 SAR data (split into two lines because it is very long)
train_sentinel = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(train_aoi).filterDate('2019-01-01', '2019-12-31')
# check which bands are available in aoi in the most recent image (HH and HV are not available, maybe due to the direction the satellite is pointing)
print("Sentinel-1 bands: " ,train_sentinel.sort('system:time_start', False).first().clip(train_aoi).bandNames().getInfo()) 
train_sentinel = train_sentinel.select('VV')



# Landsat 8 data: we only use 2019 data
train_landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(train_aoi).filterDate('2019-01-01', '2019-12-31')

# function to compute NDVI
def calculate_NDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# compute NDVI
train_landsat_ndvi = train_landsat.map(calculate_NDVI).select('NDVI')


PALSAR bands:  ['HH', 'HV', 'angle', 'date', 'qa']
Sentinel-1 bands:  ['VV', 'VH', 'angle']


# Folium functions

In [6]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [7]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    #try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    #except:
    #    print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [8]:
#converts rgb value to hex
def rgb(r, g, b):
  return ('#%02x%02x%02x' % (r, g, b))

# Visualize the training set map

In [55]:
# Get one image from the deforestation year collection, and remove values where no deforestation has occurred
train_deforestation_year_image = train_deforestation_year.first().clip(train_aoi)
train_deforestation_year_image = train_deforestation_year_image.updateMask(train_deforestation_year_image.neq(0))

# Convert to folium coordinates
train_location = train_aoi.centroid().coordinates().getInfo()[::-1]

# Colour parameters for deforestation_year
vis_params = {
'min': 1982,
'max': 2020,
#"palette": ["#2892c7", "#b30800"]}
"palette": [rgb(40,146,199), rgb(179,8,0)]}

# Create the map object.
f = folium.Figure(width=1000, height=500)
m = folium.Map(location=train_location, zoom_start=12).add_to(f)

# add satellite imagery
basemaps['Google Satellite Hybrid'].add_to(m)

# Add the deforestation year to the map
m.add_ee_layer(train_deforestation_year_image, vis_params, 'Deforestation Year')



# Get one image from the intensity collection
train_intensity_image = train_intensity.first().clip(train_aoi)

# Colour parameters for intensity
vis_params = {
'min': 1,
'max': 500,
"palette": [rgb(0,128,0), rgb(255,0,0)]}

# Add the intensity to the map
m.add_ee_layer(train_intensity_image, vis_params, 'Intensity')




# Get one image from the transition map collection
train_transition_map_image = train_transition_map.clip(train_aoi)

# Colour parameters for transition map
vis_params = {
"palette": [rgb(0,80,0), rgb(210,250,60 ), rgb(255,230,100 ), rgb(0,70,160 ), rgb(255,255,255)]}

# Add the transition map to the map
m.add_ee_layer(train_transition_map_image, vis_params, 'Transition Map')




# Get one image from the 2019 landsat NDVI collection with the least clouds
train_landsat_ndvi_image = train_landsat_ndvi.sort('CLOUD_COVER').first().clip(train_aoi) #2019 image is not blank, but when using .filterDate('2014-01-01', '2020-01-01') it was blank... why?

# Colour parameters for NDVI
vis_params = {
    'min': -1,
    'max': 1,
    "palette": [rgb(255, 255, 204), rgb(0, 153, 0)]}

# Add the NDVI to the map
m.add_ee_layer(train_landsat_ndvi_image, vis_params, 'Landsat NDVI')



# Get one image of the most recent PALSAR data
train_palsar_image = train_palsar.sort('system:time_start', False).first().clip(train_aoi)

# Colour parameters for PALSAR
vis_params = {
    'min': 0,
    'max': 10000,}

# Add the PALSAR to the map
m.add_ee_layer(train_palsar_image, vis_params, 'PALSAR')




# Get one image of the most recent Sentinel data
# train_sentinel_image = train_sentinel.sort('system:time_start', False).first().clip(train_aoi)
train_sentinel_image = train_sentinel.mosaic().clip(train_aoi)

# Colour parameters for Sentinel
vis_params = {
    'min': -50,
    'max': 1,}

# Add the Sentinel to the map
m.add_ee_layer(train_sentinel_image, vis_params, 'Sentinel-1')




# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
f

# Visualize the test set map

In [60]:
# Get one image from the deforestation year collection, and remove values where no deforestation has occurred
test_deforestation_year_image = test_deforestation_year.first().clip(test_aoi)
test_deforestation_year_image = test_deforestation_year_image.updateMask(test_deforestation_year_image.neq(0))

# Convert to folium coordinates
test_location = test_aoi.centroid().coordinates().getInfo()[::-1]

# Colour parameters for deforestation_year
vis_params = {
'min': 1982,
'max': 2020,
#"palette": ["#2892c7", "#b30800"]}
"palette": [rgb(40,146,199), rgb(179,8,0)]}

# Create the map object.
f = folium.Figure(width=1000, height=500)
m = folium.Map(location=test_location, zoom_start=12).add_to(f)

# add satellite imagery
basemaps['Google Satellite Hybrid'].add_to(m)

# Add the deforestation year to the map
m.add_ee_layer(test_deforestation_year_image, vis_params, 'Deforestation Year')



# Get one image from the intensity collection
test_intensity_image = test_intensity.first().clip(test_aoi)

# Colour parameters for intensity
vis_params = {
'min': 1,
'max': 500,
"palette": [rgb(0,128,0), rgb(255,0,0)]}

# Add the intensity to the map
m.add_ee_layer(test_intensity_image, vis_params, 'Intensity')




# Get one image from the transition map collection
test_transition_map_image = test_transition_map.clip(test_aoi)

# Colour parameters for transition map
vis_params = {
"palette": [rgb(0,80,0), rgb(210,250,60 ), rgb(255,230,100 ), rgb(0,70,160 ), rgb(255,255,255)]}

# Add the transition map to the map
m.add_ee_layer(test_transition_map_image, vis_params, 'Transition Map')




# Get one image from the 2019 landsat NDVI collection with the least clouds
test_landsat_ndvi_image = test_landsat_ndvi.sort('CLOUD_COVER').first().clip(test_aoi) #2019 image is not blank, but when using .filterDate('2014-01-01', '2020-01-01') it was blank... why?

# Colour parameters for NDVI
vis_params = {
    'min': -1,
    'max': 1,
    "palette": [rgb(255, 255, 204), rgb(0, 153, 0)]}

# Add the NDVI to the map
m.add_ee_layer(test_landsat_ndvi_image, vis_params, 'Landsat NDVI')



# Get one image of the most recent PALSAR data
test_palsar_image = test_palsar.sort('system:time_start', False).first().clip(test_aoi)

# Colour parameters for PALSAR
vis_params = {
    'min': 0,
    'max': 10000,}

# Add the PALSAR to the map
m.add_ee_layer(test_palsar_image, vis_params, 'PALSAR')




# Get one image of the most recent Sentinel data
test_sentinel_image = test_sentinel.sort('system:time_start', False).first().clip(test_aoi)

# Colour parameters for Sentinel
vis_params = {
    'min': -50,
    'max': 1,}

# Add the Sentinel to the map
m.add_ee_layer(test_sentinel_image, vis_params, 'Sentinel-1')




# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
f

# Prepare images for training/testing

In [62]:
# For training/testing data, we only use the SAR + NDVI
# Also, we do normalization here based on nominal min/max values of each satellite, though it probably should have been done earlier during data loading
train_img = train_palsar_image.rename("PALSAR").unitScale(0,10000)
train_img = train_img.addBands(train_sentinel_image.rename("Sentinel").unitScale(-50,1))
train_img = train_img.addBands(train_landsat_ndvi_image.rename("NDVI").unitScale(-1,1))

# A simplified image for debugging
train_img_debug = train_palsar_image.rename("PALSAR").unitScale(0,10000) 
train_img_debug = train_img_debug.addBands(train_landsat_ndvi_image.rename("NDVI").unitScale(-1,1))
train_img_debug = train_img_debug.addBands(train_sentinel_image.rename("Sentinel").unitScale(-50,1))

# Add a band to the training data that contains the transition map (i.e. the class label)
train_img = train_img.addBands(train_transition_map)
train_img_debug = train_img_debug.addBands(train_transition_map)

test_img = test_palsar_image.rename("PALSAR").unitScale(0,10000).addBands([test_sentinel_image.rename("Sentinel").unitScale(-50,1), test_landsat_ndvi_image.rename("NDVI").unitScale(-1,1)])
test_img_debug = test_landsat_ndvi_image.rename("PALSAR").unitScale(-1,1)

# convert classes to a FeatureCollection so they can be used by supervised algorithms
#train_classes = ee.FeatureCollection(train_transition_map)

#test_classes = ee.FeatureCollection(test_transition_map)

# visualize to make sure it is working correctly

f = folium.Figure(width=1000, height=500)
m = folium.Map(location=train_location, zoom_start=12).add_to(f)

vis_params = {
    'bands': ["Sentinel"],
    'min': 0,
    'max': 1,}

m.add_ee_layer(train_img, vis_params, "Training Image")

f



# Supervised learning

Training

In [57]:
# PALSAR is 25m, Sentinel is 10m, NDVI from Landat8 is 30m, and the Vancutsem transition map is also 30m, so we use a scale of 30m for simplicity
# Sample 1000 points from the training dataset
train_points = train_img.sample(numPixels = 10000, 
                                scale = 30, 
                                seed = 0)

train_points_debug = train_img_debug.sample(numPixels = 1000, 
                                            scale = 30, 
                                            seed = 0)

rf_classifier = ee.Classifier.smileRandomForest(numberOfTrees = 10).train(features = train_points,
                                                                          classProperty = 'class',
                                                                          inputProperties = ["PALSAR", "Sentinel", "NDVI"])

rf_classifier_debug = ee.Classifier.smileRandomForest(numberOfTrees = 10).train(features = train_points,
                                                                                classProperty = 'class',
                                                                                inputProperties = ["PALSAR", "NDVI"])




In [58]:
train_points_debug = train_img.sample(numPixels = 1, 
                                            scale = 30, 
                                            seed = 0)

train_points_debug.first().getInfo()

{'geometry': None,
 'id': '0',
 'properties': {'NDVI': 0.7092303937102096,
  'PALSAR': 0.3252,
  'Sentinel': 0.6838712119589899,
  'class': 4},
 'type': 'Feature'}

In [50]:
train_points_debug = train_img_debug.sample(numPixels = 1, 
                                            scale = 30, 
                                            seed = 0)

train_points_debug.first().getInfo()

Testing

In [59]:
classified_test_img = test_img.classify(rf_classifier).clip(test_aoi)

classified_test_img_debug = test_img_debug.classify(rf_classifier_debug).clip(test_aoi)

Visualize predicted classes against true classes

In [61]:
f = folium.Figure(width=1000, height=500)
m = folium.Map(location=test_location, zoom_start=12).add_to(f)

test_transition_map_image = test_transition_map.clip(test_aoi)

vis_params = {
"palette": [rgb(0,80,0), rgb(210,250,60), rgb(255,230,100), rgb(0,70,160 ), rgb(255,255,255)]}

m.add_ee_layer(test_transition_map_image, vis_params, 'True Transition Map')
m.add_ee_layer(classified_test_img, vis_params, 'Predicted Transition Map')
m.add_child(folium.LayerControl())

f

Visualize predicted classes against true classes (debug)

In [14]:
f = folium.Figure(width=1000, height=500)
m = folium.Map(location=test_location, zoom_start=12).add_to(f)

test_transition_map_image = test_transition_map.clip(test_aoi)

vis_params = {
"palette": [rgb(0,80,0), rgb(210,250,60), rgb(255,230,100), rgb(0,70,160 ), rgb(255,255,255)]}

m.add_ee_layer(test_transition_map_image, vis_params, 'True Transition Map')
m.add_ee_layer(classified_test_img_debug, vis_params, 'Predicted Transition Map')
m.add_child(folium.LayerControl())

f

# Unsupervised learning